In [19]:
import numpy as np
import matplotlib.pyplot as plt


class Function:
    def __init__(self, value):
        self.value = value
        self.grad = lambda x: x


def grad_desc(func, lr, x, eps):
    points = [x]

    while True:
        prev_x = x
        x = x - lr(Function(lambda a: func.value(x - a * func.grad(x)))) * func.grad(x)
        points.append(x)
        if abs(func.value(x) - func.value(prev_x)) < eps:
            break

    return np.array(points)


def right_border_calc(func):
    right_start = 0.25
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt):
    while abs(a_1 - a_2) >= eps:
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt

        if func.value(new_a_2) > func.value(new_a_1):
            a_2 = new_a_2
        else:
            a_1 = new_a_1

    return (a_1 + a_2) / 2


In [ ]:

def func(x):
    return np.array((x[0] - 4) ** 2 + (5 - x[1]) ** 2)


def lr_wrapper(eps, delt):
    return lambda func: dichotomy(func, 0, right_border_calc(func), eps, delt)


print(grad_desc(Function(func), lr_wrapper(0.01, 0.0002), np.array((20, 20)), 0.001))

C:\Users\vryab\AppData\Local\Temp\ipykernel_12944\272492716.py:16: RuntimeWarning: invalid value encountered in multiply
  x = x - lr(Function(lambda a: func.value(x - a * func.grad(x)))) * func.grad(x)
